In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from IPython.display import display

In [2]:

def get_valid_days(df_month, max_days=2):
    """
    Retourne les deux premiers jours valides parmi 13, 12, 11, 10
    
    Args:
        df_month: DataFrame contenant les données d'un mois
        max_days: Nombre maximum de jours à retourner (par défaut: 2)
        
    Returns:
        list: Liste des jours valides (maximum max_days)
    """
    valid_days = []
    days_to_check = [13, 12, 11, 10]  # Jours à vérifier
    
    unique_days = df_month['Date'].dt.day.unique()
    
    for day in days_to_check:
        if day in unique_days:
            valid_days.append(day)
            if len(valid_days) >= max_days:
                break
                
    return valid_days

In [3]:
input_file = "TBA_database_reduced.csv"  # Remplacer par votre chemin d'entrée
output_file = "net_basis_results.csv"    # Remplacer par votre chemin de sortie

df = pd.read_csv(input_file)

In [4]:
df.replace('#N/A N/A', np.nan, inplace=True)

# Convertir la colonne Date au format datetime si nécessaire
if not pd.api.types.is_datetime64_any_dtype(df['Date']):
    df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%y')

df['CPR_Dynamic'] = 10 #df.apply(lambda row: calculate_cpr(row['Coupon'], row['Mortgage rate']), axis=1)

# Liste pour stocker les résultats
results = []

# Dictionnaire pour stocker les résultats temporaires (pour le lissage sur deux jours)
temp_results = {}

# Regrouper par mois pour le traitement des jours valides
df_by_month = df.groupby(df['Date'].dt.to_period('M'))

print("Traitement des données par mois...")

Traitement des données par mois...


In [29]:
period, df_month  = list(iter(df_by_month))[1]

In [30]:
period

Period('2015-02', 'M')

In [32]:
# Déterminer les jours valides pour ce mois
valid_days = get_valid_days(df_month, max_days=2)

valid_days

roll_date = valid_days[0]

# Filtrer les données pour ce jour
df_roll_date = df_month[df_month['Date'].dt.day == roll_date]


row = df_roll_date.iloc[0]

date = row['Date']
repo_rate = row['Real repo']

# Déterminer la CPR à utiliser
coupon_value = row['Coupon']
cpr_column = f"CPR Realized {coupon_value}"

if cpr_column in df.columns and pd.notna(row[cpr_column]):
    cpr_realized = row[cpr_column]
else:
    cpr_realized = row['CPR_Dynamic']

cpr_model = row['CPR_Dynamic']

# Traiter les mois disponibles
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [35]:
# 1. Traiter le net basis "current"
for i in range(len(months) - 1):
    current_month = months[i]
    next_month = months[i + 1]
    
    if pd.notna(row[current_month]) and pd.notna(row[next_month]):
        print(current_month)
        break

Feb


In [ ]:

# Convertir les prix en décimal
current_price = float(row[current_month]) / 100
next_price = float(row[next_month]) / 100

date, current_price, coupon_value, repo_rate, cpr_realized, roll_date


(Timestamp('2015-02-13 00:00:00'),
 1.13551,
 np.float64(6.0),
 np.float64(0.12),
 np.float64(0.23),
 13)

In [ ]:
df['Real repo']

0        0.12
1        0.12
2        0.12
3        0.12
4        0.12
         ... 
23185    4.33
23186    4.33
23187    4.33
23188    4.33
23189    4.33
Name: Real repo, Length: 23190, dtype: float64

: 